<a href="https://colab.research.google.com/github/LTech365/Alura-IA1/blob/main/Agente_Relatorio_MCMV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
# Instalar bibliotecas necessárias
%pip -q install google-genai google-adk

In [95]:
import os
from google.colab import userdata
from IPython.display import display, Markdown
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
import textwrap
import warnings
import json # Pode ser útil para dados

warnings.filterwarnings("ignore") # Ignora alguns warnings que podem aparecer

In [96]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [97]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [98]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final.
def call_agent_simple(agent: Agent, user_input: str) -> str:
    session_service = InMemorySessionService()
    # Usa um session_id genérico, pois é uma única chamada
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="single_call")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    content = types.Content(role="user", parts=[types.Part(text=user_input)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="single_call", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [99]:
# Função auxiliar para exibir texto formatado em Markdown no Colab.
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [100]:
"""### Definição do Agente Gerador de Relatório"""

# --- Agente Único: Agente Gerador de Relatório ---
# Este agente recebe os dados coletados pelo código Python e gera um relatório sumarizado.
# Não há Agente Entrevistador neste modelo, o código Python faz a entrevista.
def agente_gerador_relatorio_simplificado():
    return Agent(
        name="agente_gerador_relatorio",
        model=MODEL_ID, # Usa o modelo definido
        description="Agente que recebe dados coletados de uma entrevista controlada pelo código e gera um relatório sumarizado.",
         # Este agente não precisa de ferramentas externas.
        instruction="""
        Você é um agente especializado em gerar relatórios de dados coletados de entrevistas.
        Receberá dados de um cliente no formato "Campo: Resposta".
        Sua tarefa é organizar esses dados em um relatório claro e conciso, adequado para um analista humano revisar.
        Liste cada dado coletado com seu respectivo valor de forma organizada.
        """
    )

In [101]:
"""### Execução do Sistema (Entrevista Controlada pelo Código e Geração de Relatório)"""

# --- Lógica Principal: Controla a Entrevista e Chama o Agente para o Relatório ---

print("🏠 Iniciando o Sistema Simplificado de Coleta de Dados para Pré-Qualificação 🏠")
print("Por favor, responda às perguntas uma por vez.")

# Dicionário para armazenar as respostas do cliente (ficha)
ficha_cliente = {}

# --- Sequência de Perguntas Controlada pelo Código Python ---
# Definimos as perguntas aqui, e o código as apresentará uma por uma.
perguntas_entrevista = {
    "briefing": "Olá! Tudo bem? Qual o motivo da compra do imóvel: moradia ou investimento?",
    "data_nascimento": "Qual a sua data de nascimento, por favor? (formato dd/mm/aaaa)",
    "renda": "Qual a sua renda bruta familiar mensal? Considere a renda regular, sem incluir 13º salário ou bônus de férias.",
    "fgts": "Você já trabalhou de carteira assinada por 3 anos ou mais (não precisa ser consecutivo)? (Responda Sim ou Não)",
    "propriedade_imovel": "Você é proprietário de algum imóvel residencial que esteja QUITADO e registrado em seu nome no Registro Geral de Imóveis (RGI) da cidade? Lembre-se: terreno, lote ou imóvel financiado em outro banco NÃO é impeditivo para o MCMV. (Responda Sim ou Não)",
    "beneficio_anterior": "Você já recebeu algum benefício habitacional anterior do governo federal? (Responda Sim ou Não)"
}

print("\n--- Iniciando Entrevista ---")

# Loop para fazer as perguntas sequencialmente
for campo, pergunta in perguntas_entrevista.items():
    print(f"Atendente: {pergunta}")
    # Coleta a resposta do usuário diretamente no código Python
    resposta_usuario = input("Você: ")
    # Armazena a resposta na ficha
    ficha_cliente[campo] = resposta_usuario
    print("-" * 20) # Separador para clareza na saída

print("\n--- Entrevista Concluída ---")
print("Dados coletados:")
# Exibe os dados coletados pelo código Python
print(ficha_cliente)


# --- Etapa 2: Executar o Agente Gerador de Relatório ---
# Esta etapa ocorre APENAS após a entrevista ter sido concluída pelo código Python.

if ficha_cliente: # Verifica se algum dado foi coletado
    print("\n--- Gerando Relatório dos Dados Coletados ---")

    # Obtenha o Agente Gerador de Relatório
    agente_relatorio = agente_gerador_relatorio_simplificado()

    # Prepara a entrada para o Agente do Relatório. Passamos os dados coletados como uma string formatada.
    dados_para_relatorio = "\n".join([f"{k.replace('_', ' ').title()}: {v}" for k, v in ficha_cliente.items()])
    entrada_agente_relatorio = f"Dados do Cliente Coletados:\n{dados_para_relatorio}"

    # Executa o Agente do Relatório para gerar o relatório
    relatorio_gerado = call_agent_simple(agente_relatorio, entrada_agente_relatorio)

    print("\n--- Relatório Final dos Dados Coletados ---")
    # Exibe o relatório gerado pelo Agente 2 (simulando o conteúdo do arquivo a ser salvo)
    display(to_markdown(relatorio_gerado))
else:
    print("\n--- Nenhum dado coletado para gerar relatório. ---")


print("\n✅ Processo de Coleta de Dados e Geração de Relatório Concluído.")

🏠 Iniciando o Sistema Simplificado de Coleta de Dados para Pré-Qualificação 🏠
Por favor, responda às perguntas uma por vez.

--- Iniciando Entrevista ---
Atendente: Olá! Tudo bem? Qual o motivo da compra do imóvel: moradia ou investimento?
Você: Moradia
--------------------
Atendente: Qual a sua data de nascimento, por favor? (formato dd/mm/aaaa)
Você: 03/06/1978
--------------------
Atendente: Qual a sua renda bruta familiar mensal? Considere a renda regular, sem incluir 13º salário ou bônus de férias.
Você: 5000,00
--------------------
Atendente: Você já trabalhou de carteira assinada por 3 anos ou mais (não precisa ser consecutivo)? (Responda Sim ou Não)
Você: Sim
--------------------
Atendente: Você é proprietário de algum imóvel residencial que esteja QUITADO e registrado em seu nome no Registro Geral de Imóveis (RGI) da cidade? Lembre-se: terreno, lote ou imóvel financiado em outro banco NÃO é impeditivo para o MCMV. (Responda Sim ou Não)
Você: Não
--------------------
Atendente:

> Relatório de Dados do Cliente:
> 
> Briefing: Moradia
> Data de Nascimento: 03/06/1978
> Renda: R$ 5000,00
> FGTS: Sim
> Propriedade de Imóvel: Não
> Benefício Anterior: Não



✅ Processo de Coleta de Dados e Geração de Relatório Concluído.
